In [1]:
import os
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
import requests
import traceback
import pandas as pd
from io import BytesIO

from utils import standardizeFuels, standardizeFields, createJoinKey, isURLValid

In [2]:
os.chdir('../../gi-queue-map/')
print(os.getcwd())

c:\Users\zleig\Documents\GitHub\gi-queue-map


In [3]:
def findNewURL(utility):
    #Access the download settings that track working urls and data updates
    ds_path = f'scripts/script_data/download_settings.csv'
    download_settings = pd.read_csv(ds_path, index_col='name')

    #Read data from download settings
    base_url = download_settings.loc[utility]['base_url']
    last_updated = '12/11/2024'
    #last_updated = download_settings.loc[utility]['last_updated']
    print(last_updated)
    date_format = download_settings.loc[utility]['date_format']

    #Convert tracker to datetime object
    date_tracker = datetime.strptime(last_updated, "%m/%d/%Y")

    #If the URL on file is already working, return None to indicate no update is necessary
    if isURLValid(base_url.format(date_tracker.strftime(date_format))):
        print("URL is still valid")
        return None
    print("Looking for new url")
    #If the URL on file does not work anymore, loop through all possible dates to find something new
    while date_tracker < datetime.now():
        #Create URL for testing
        formatted_date = date_tracker.strftime(date_format)
        full_url = base_url.format(formatted_date)

        #If the URL is valid, that means that the dataset needs to be updated
        if isURLValid(full_url):
            correct_date = date_tracker.strftime("%m/%d/%Y")
            #Update download settings to reflect changes
            download_settings.loc[utility, 'last_updated'] = correct_date
            #Save changes
            download_settings.to_csv(ds_path)
            #Return the new URL so the module can update the data
            return full_url
        else:
            #If no valid URL found, try the next day
            date_tracker = date_tracker + timedelta(days=1)
    print("Attention needed for " + utility, "No valid link found")
    return None



In [4]:
def importDuke(url):
    url = url
    response = requests.get(url)
    excel_file = BytesIO(response.content)
    # The active projects are in the first sheet
    # Gets read by default
    duke_df = pd.read_excel(excel_file, engine='openpyxl')
    header_row_index = duke_df[duke_df.iloc[:, 1] == "OPCO"].index[0]

    # Set that row as the header
    df_cleaned = duke_df.iloc[header_row_index + 1:-2].reset_index(drop=True)  # Rows below the header row
    df_cleaned.columns = duke_df.iloc[header_row_index]  # Set the column names
    return df_cleaned

In [5]:
dep_url = findNewURL('DEP')
dec_url = findNewURL('DEC')

dep_df = importDuke(dep_url)
dec_df = importDuke(dec_url)

duke_df = pd.concat([dec_df, dep_df])
#duke_df

12/11/2024
Looking for new url
12/11/2024
Looking for new url


In [9]:
duke_df = duke_df[~duke_df['Operational Status'].isin(["Withdrawn", "Commercial Operation - Commercial Operation Date Declared"])]

### Pretend like this is a standalone function

In [14]:
def importDuke(url):
    response = requests.get(url)
    excel_file = BytesIO(response.content)
    # The active projects are in the first sheet
    # Gets read by default
    duke_df = pd.read_excel(excel_file, engine='openpyxl')
    header_row_index = duke_df[duke_df.iloc[:, 1] == "OPCO"].index[0]

    # Set that row as the header
    df_cleaned = duke_df.iloc[header_row_index + 1:-2].reset_index(drop=True)  # Rows below the header row
    df_cleaned.columns = duke_df.iloc[header_row_index]  # Set the column names
    return df_cleaned

def getDukeQueue():
    dep_url = findNewURL('DEP')
    dec_url = findNewURL('DEC')

    if (dep_url is None) or (dec_url is None):
        duke_backup = pd.read_csv('data/individual_queues/duke_active_projects.csv')
        return duke_backup
    else:
        dep_df = importDuke(dep_url)
        dec_df = importDuke(dec_url)

        duke_df = pd.concat([dec_df, dep_df])

        ### Begin clean up ###
        duke_df = duke_df[~duke_df['Operational Status'].isin(["Withdrawn", "Commercial Operation - Commercial Operation Date Declared"])]
        duke_df['POI'] = duke_df['Transmission Line'].astype(str) + duke_df['Substation Name'].astype(str)
        return duke_df
getDukeQueue()


12/11/2024
Looking for new url
12/11/2024
Looking for new url


,NaN,OPCO,Transmission Designation,Source System Unique ID,Queue Number,Queue Issued Date,Interconnection Customer,Queue Indicator*,Operational Status,Operational Date,...,Project Size Winter MWAC,Energy Source Type,Facility County,Facility State,Transmission Line,Substation Name,Duke Estimated Startup Date,Type of Service,Advance Notice of Scoping Meeting,POI
0,NaN,DEC,FERC,962624,2024-09-27 14:15:00,2024-09-27 00:00:00,Masked,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,74.9,Solar,Greenwood,SC,Florida 44 kV,NaN,NaN,Both,NaN,Florida 44 kVnan
1,NaN,DEC,FERC,963122,2024-09-26 10:42:00,2024-09-26 00:00:00,Duke Energy Carolinas LLC,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,50,Solar,Laurens,SC,NaN,NaN,NaN,NRIS,NaN,nannan
2,NaN,DEC,FERC,962913,2024-09-26 12:05:00,2024-09-26 00:00:00,Masked,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,120000,Solar,McCormick,SC,NaN,NaN,NaN,NRIS,NaN,nannan
3,NaN,DEC,FERC,962989,2024-09-24 11:23:00,2024-09-24 00:00:00,Duke Energy Carolinas LLC,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,34,Solar,Anderson,SC,NaN,NaN,NaN,NRIS,NaN,nannan
4,NaN,DEC,FERC,962640,2024-09-24 11:30:00,2024-09-24 00:00:00,Duke Energy Carolinas LLC,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,130,Solar,Stokes,NC,NaN,NaN,NaN,NRIS,NaN,nannan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,NaN,DEP,FERC,049220,2019-10-08 11:39:00,2019-10-08 00:00:00,"Duke Energy Progress, LLC",Transitional Serial Study,Engineering Design - In Progress,NaN,...,17.25,Battery,Buncombe,NC,NaN,NaN,2026-03-31 00:00:00,NaN,NaN,nannan
98,NaN,DEP,FERC,038056,2019-06-20 14:45:00,2019-06-20 00:00:00,"Knightdale Energy Center, LLC",Transitional Serial Study,Engineering Design - Pending,NaN,...,100,Battery,Wake,NC,Milburnie - Wake 230 kV,NaN,NaN,Both,NaN,Milburnie - Wake 230 kVnan
118,NaN,DEP,FERC,018124,Q442,2017-10-26 00:00:00,"Duke Energy Progress, LLC",Transitional Serial Study,Commissioning - Permission to Operate at Full ...,2023-03-01 00:00:00,...,11,Battery,Onslow,NC,NaN,NaN,NaN,NaN,NaN,nannan
126,NaN,DEP,FERC,948720,2017-06-16 00:00:00,2017-06-16 00:00:00,"Duke Energy Progress, LLC",NaN,Construction - In Progress,NaN,...,80000,Solar,Nash,NC,NaN,NaN,2025-07-31 00:00:00,NRIS,NaN,nannan
